In [ ]:
import pandas as pd

bim_file = "dataset/dog10k.snps.plink.bim"
gff_file = "dataset/GCF.gff"

bim_cols = ["chrom", "snp_id", "gen_dist", "pos", "allele1", "allele2"]
bim_df = pd.read_csv(bim_file, sep=r"\s+", names=bim_cols)

gff_cols = ["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"]
gff_genes = pd.read_csv(
    gff_file,
    sep="\t",
    comment="#",
    names=gff_cols,
    usecols=["seqid", "type", "start", "end", "attributes"]
)
gff_genes = gff_genes[gff_genes["type"].isin(["gene", "pseudogene"])].copy()

Loaded Dataset


In [2]:
def extract_gene_name(attr):
    for key in ["gene", "Name"]:
        if f"{key}=" in attr:
            return attr.split(f"{key}=")[1].split(";")[0]
    return None

def map_snp_to_gene(row, gff_genes):
    chrom = str(row["chrom"])
    pos = row["pos"]
    genes_on_chrom = gff_genes[gff_genes["seqid"] == chrom]
    hits = genes_on_chrom[(genes_on_chrom["start"] <= pos) & (genes_on_chrom["end"] >= pos)]
    if not hits.empty:
        return ";".join(hits["gene_name"].tolist())
    else:
        return None

gff_genes["gene_name"] = gff_genes["attributes"].apply(extract_gene_name)
bim_df["gene"] = bim_df.apply(map_snp_to_gene, axis=1, gff_genes=gff_genes)

KeyboardInterrupt: 

In [ ]:
bim_df.to_csv("snp_to_gene_mapping.csv", index=False)